In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os

/Users/tyronemariano/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
#initialize image resolution
width = 256
height = 256

In [ ]:
#load the best model
best_model_file = os.path.expanduser("~/Desktop/lung-Unet.keras")
model = tf.keras.models.load_model(best_model_file)
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 94,183,941 (359.28 MB)

 Trainable params: 31,390,721 (119.75 MB)

 Non-trainable params: 11,776 (46.00 KB)

 Optimizer params: 62,781,444 (239.49 MB)

None


In [ ]:
input_folder = "data"  #set-up the path to the input folder; this is where we put SAN LAZARO IMAGES
output_folder = "segmented" #set-up the path to the output folder
os.makedirs(output_folder, exist_ok=True)

In [ ]:
for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  #ensure processing only image files
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        
        if img is None:
            print(f"Skipping {filename}, unable to read image.")
            continue
        
        img_resized = cv2.resize(img, (width, height)) / 255.0
        img_for_model = np.expand_dims(img_resized, axis=0)
        
        #predict mask
        p = model.predict(img_for_model)
        result_mask = p[0]
        
        #binarize mask
        result_mask[result_mask <= 0.5] = 0
        result_mask[result_mask > 0.5] = 255
        
        #resize mask back to original dimensions
        mask_resized = cv2.resize(result_mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_AREA)
        mask_resized = mask_resized.astype(np.uint8)
        
        #convert to 3-channel image
        mask_bgr = cv2.cvtColor(mask_resized, cv2.COLOR_GRAY2BGR)
        
        #apply mask
        segmented_image = cv2.bitwise_and(img, mask_bgr)
        
        #save the segmented image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, segmented_image)
        print(f"Processed and saved: {output_path}")

print("Segmentation complete.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step
Processed and saved: subsegmented/TB_325_1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
Processed and saved: subsegmented/Normal_678_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step
Processed and saved: subsegmented/TB_360_1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
Processed and saved: subsegmented/Normal_110_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
Processed and saved: subsegmented/TB_408_1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step
Processed and saved: subsegmented/Normal_155_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
Processed and saved: subsegmented/Normal_584_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
Processed and saved: subsegmented/Normal_579_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
Processed and saved: subsegmented/Normal_211_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step
Processed and saved: subsegmented/Normal_685_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
Processed and saved: subsegmented/Normal_254_0.png
1/1 ━━━━━━━━━━━━━━━━━━